In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features

fig_dir='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Saildrone/'

data_dir = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission/'

adir_sbe='F:/data/cruise_data/saildrone/2019_arctic/sbe56/sd-'

data_dir_sbe_combined = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission_combined/'

# Metadata
- SBE56 info, set some attributes and depths for metadata
- put filenames and depts into a xarray dataset


In [ ]:

vattrs = {'long_name': 'sea surface depth','coordinates': 'time',
          'standard_name': 'sea_water_temperature', 'installed_height':'-0.295 m' ,
          'serial_number' : 'SBE05608196', 'model_number': '56', 
          'sensor_description': 'Seabird 56 Temperature Logger',
          'model_name': 'SBE 56', 'product_page':'http://www.seabird.com/sbe56-temperature-logger', 
          'nominal_sampling_schedule': '2 sec', 
          'units':'degrees_C','valid_min':-10.0,'valid_max':50.0}


astr_sbe36=['SBE05609153_2019-11-25.csv','SBE05609154_2019-11-25.csv',
         'SBE05609158_2019-11-25.csv','SBE05609150_2019-11-25.csv',
         'SBE05609151_2019-11-25.csv','SBE05609152_2019-11-25.csv',
           'SBE05609157_2019-11-25.csv']

depth36=[330,473,823,1048,1209,1436,1700]

astr_sbe37=['SBE05608196_2019-11-25.csv','SBE05604705_2019-11-25.csv',
         'SBE05605200_2019-11-25.csv','SBE05608476_2019-11-25.csv',
         'SBE05609159_2019-11-25.csv','SBE05609155_2019-11-26.csv',
         'SBE05609156_2019-11-26.csv']
depth37=[324,474,805,1035,1196,1413,1725]

adepth=[depth36,depth37]
astr = [astr_sbe36,astr_sbe37]

ds_info = xr.Dataset(data_vars={'fname':(['trajectory','z'],astr),
                                'depth':(['trajectory','z'],adepth)},
                     coords={'z':np.arange(7),'trajectory':[1036,1037]})




# Combine the .cvs files into a single netcdf file with metadata

In [ ]:
for itt in range(0,2):
    for iz  in range(7):
        if (itt==0) and (iz==3):  #the third sbe56 fell off & file doesn't exist
            continue
        fstr = str(ds_info.fname[itt,iz].data)
        avehicle=str(ds_info.trajectory[itt].data)
        filename=adir_sbe+avehicle+'/'+fstr
        print(filename)
        df = pd.read_csv(filename,header=11) #, sheet_name='data')
        df['time']=pd.to_datetime(df['Date'] + ' ' + df['Time']) 
        del df['Date']
        del df['Time']
        df.index=df['time']
        xf=df.to_xarray()
        vname='sea_water_temperature_'+str(iz).zfill(2)
        xf=xf.rename({'Temperature':vname})
        vattrs['installed_height']=str(-1*ds_info.depth[itt,iz].data/1000.)+' m'
        vattrs['serial_number']=fstr[:11]
        xf.attrs=vattrs
        if iz==0:
            ds_sbe=xr.Dataset({'sea_water_temperature_00':xf[vname]})
            ds_sbe[vname].attrs=vattrs
        else:
            ds_sbe=ds_sbe.assign(vname=xf[vname])
            ds_sbe=ds_sbe.rename({'vname':vname})
            ds_sbe[vname].attrs=vattrs
    if itt==0:
        ds_sbe['time']=ds_sbe.time+np.timedelta64(8,'h')
    fout=adir_sbe+'combined'+avehicle+'.nc'
    ds_sbe.to_netcdf(fout)

# Create files with 1 minute averages of data using :54 to :06 seconds 

In [ ]:
#make 1min average data

for itt in range(2):
    avehicle=str(ds_info.trajectory[itt].data)
    filename=adir_sbe+'combined'+avehicle+'.nc'
    filename_out=adir_sbe+'combined_1minave'+avehicle+'.nc'
    xf=xr.open_dataset(filename)
    for iz  in range(7):
        if (itt==0) and (iz==3):
            continue
        print(itt,iz)
        vname='sea_water_temperature_'+str(iz).zfill(2)
        vnamemn='sea_water_temperature_'+str(iz).zfill(2)+'_mean'
        vnamestd='sea_water_temperature_'+str(iz).zfill(2)+'_std'
        offset = pd.to_timedelta(6, unit='s') 
        xftem=xf[vname]
        pt = pd.to_datetime(xftem.time.data)+offset  #add 6 seconds
        psec=pt.second
        tem=xftem[(psec <=12)]  #data average should be only use sec 54 to sec 06 of each minute 
        tem['time']=tem['time']+offset #resample goes from :00 to :59 so shift +6sec
        xf_avg1 = tem.resample(time='1min').mean(keep_attrs=True)
        xf_std1 = tem.resample(time='1min').std(ddof=1,keep_attrs=True)
        sattrs=xf[vname].attrs
        sattrs['nominal_sampling_schedule']='1 minute averages from :54 to :06 seconds'
        sattrs['long_name']='sea surface depth 1-min mean'
        xf_avg1.attrs=sattrs
        sattrs['long_name']='sea surface depth 1-min standard dev'
        if iz==0:
            ds_sbe=xr.Dataset({'vname':xf_avg1})
            ds_sbe=ds_sbe.rename({'vname':vnamemn})
            ds_sbe=ds_sbe.assign(vname=xf_std1)
            ds_sbe=ds_sbe.rename({'vname':vnamestd})
        else:
            ds_sbe=ds_sbe.assign(vname=xf_avg1)
            ds_sbe=ds_sbe.rename({'vname':vnamemn})
            ds_sbe=ds_sbe.assign(vname=xf_std1)
            ds_sbe=ds_sbe.rename({'vname':vnamestd})
        #ds_sbe = ds_sbe.drop(vname)
    ds_sbe2=ds_sbe.where(np.isfinite(ds_sbe.sea_water_temperature_00_mean),drop=True)
    fout=adir_sbe+'combined'+avehicle+'.nc'
    ds_sbe2.to_netcdf(filename_out)

In [ ]:
for iusv in range(2):
    if iusv==0:
        fname='saildrone-gen_5-arctic_misst_2019-sd1036-20190514T230000-20191011T183000-1_minutes-v1.1575336154680.nc'
        fname2='sd-combined_1minave1036.nc'
        fname_out='sd-combined_1minave1036_withattrs.nc'
    if iusv==1:
        fname='saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625.nc'
        fname2='sd-combined_1minave1037.nc'
        fname_out='sd-combined_1minave1037_withattrs.nc'
    ds = xr.open_dataset(data_dir+fname)
    ds.close()
    ds_sbe=xr.open_dataset(adir_sbe[:-3]+fname2)
    ds_sbe.close()
    gattrs=ds.attrs
    ds_sbe.attrs=gattrs
    ds_sbe.sel(time=slice('2019-05-14T23:00','2019-10-11T18:30'))
    ds_sbe.to_netcdf(adir_sbe[:-3]+fname_out)

In [ ]:
ds_sbe

# combine the 1-min ave SBE data with the saildrone file of all data

In [ ]:
for iusv in range(2):
    if iusv==0:
        fname='saildrone-gen_5-arctic_misst_2019-sd1036-20190514T230000-20191011T183000-1_minutes-v1.1575336154680.nc'
        fname2='sd-combined_1minave1036.nc'
    if iusv==1:
        fname='saildrone-gen_5-arctic_misst_2019-sd1037-20190514T230000-20191011T183000-1_minutes-v1.1575487464625.nc'
        fname2='sd-combined_1minave1037.nc'
    ds = xr.open_dataset(data_dir+fname)
    ds.close()
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds['time']=ds.time[0,:]
    ds = ds.swap_dims({'obs':'time'})
    #calculate some variables
    ds['wspd_MEAN']=np.sqrt(ds.UWND_MEAN**2+ds.VWND_MEAN**2)
    ds['wdir_MEAN']=np.arctan2(ds.VWND_MEAN,ds.UWND_MEAN)*180./np.pi
    ds_sbe=xr.open_dataset(adir_sbe[:-3]+fname2)
    ds_sbe.close()
    ds_tem=ds_sbe.interp(time=ds.time,method='nearest')
    ds = ds.isel(trajectory=0)
    for var in ds_tem:
        ds[var]=ds_sbe[var]
    fname_out=fname[:-3]+'-withSBE.nc'
    ds.to_netcdf(data_dir_sbe_combined+fname_out)

In [ ]:
ds = xr.open_mfdataset(data_dir_sbe_combined+'*.nc',combine='nested',concat_dim='trajectory').load()
ds